# Exercise 8

## Car Price Prediction

Predict if the price of a car is low or high

In [89]:
import pandas as pd
import numpy as np

In [90]:
%matplotlib inline
import pandas as pd

data = pd.read_csv('../datasets/dataTrain_carListings.zip')
data = data.loc[data['Model'].str.contains('Camry')].drop(['Make', 'State'], axis=1)
data = data.join(pd.get_dummies(data['Model'], prefix='M'))
data['HighPrice'] = (data['Price'] > data['Price'].mean()).astype(int)
data = data.drop(['Model', 'Price'], axis=1)

data.head()

,Year,Mileage,M_Camry,M_Camry4dr,M_CamryBase,M_CamryL,M_CamryLE,M_CamrySE,M_CamryXLE,HighPrice
15,2016,29242,0,0,0,0,1,0,0,1
47,2015,26465,0,0,0,0,1,0,0,1
85,2012,46739,0,1,0,0,0,0,0,1
141,2017,41722,0,0,0,0,0,1,0,1
226,2014,77669,0,0,0,0,0,0,1,0


In [91]:
data.shape

(13150, 10)

In [92]:
y = data['HighPrice']
X = data.drop(['HighPrice'], axis=1)

In [93]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


# Exercise 8.1

Estimate a Decision Tree Classifier Manually using the code created in the Notebook #13

Evaluate the accuracy on the testing set

In [94]:
def gini(y):
    if y.shape[0] == 0:
        return 0
    else:
        return 1 - (y.mean()**2 + (1 - y.mean())**2)

In [95]:
def gini_impurity(X_col, y, split):
    "Calculate the gain of an split k on feature j"
    
    filter_l = X_col < split
    y_l = y.loc[filter_l]
    y_r = y.loc[~filter_l]
    
    n_l = y_l.shape[0]
    n_r = y_r.shape[0]
    
    gini_y = gini(y)
    gini_l = gini(y_l)
    gini_r = gini(y_r)
    
    gini_impurity_ = gini_y - (n_l / (n_l + n_r) * gini_l + n_r / (n_l + n_r) * gini_r)
    
    return gini_impurity_

In [96]:
def best_split(X, y, num_pct=10):
    
    features = range(X.shape[1])
    
    best_split = [0, 0, 0]  # j, split, gain
    
    # For all features
    for j in features:
        
        splits = np.percentile(X.iloc[:, j], np.arange(0, 100, 100.0 / (num_pct+1)).tolist())
        splits = np.unique(splits)[1:]
        
        # For all splits
        for split in splits:
            gain = gini_impurity(X.iloc[:, j], y, split)
                        
            if gain > best_split[2]:
                best_split = [j, split, gain]
    
    return best_split

In [97]:
def tree_grow(X, y, level=0, min_gain=0.001, max_depth=None, max_features=7, num_pct=10):
    
    # If only one observation
    if X.shape[0] == 1:
        tree = dict(y_pred=y.iloc[:1].values[0], y_prob=0.5, level=level, split=-1, n_samples=1, gain=0)
        return tree
    
    #Selección de variables
    Z = np.random.choice(range(X.shape[1]),3, replace = 0)
    #print(Z)
    H = X.iloc[: ,Z]
    #print(H.head())
    
    # Calculate the best split
    j, split, gain = best_split(H, y, num_pct)
    
    
    # save tree and estimate prediction
    y_pred = int(y.mean() >= 0.5) 
    y_prob = (y.sum() + 1.0) / (y.shape[0] + 2.0)  # Laplace correction
    
    tree = dict(y_pred=y_pred, y_prob=y_prob, level=level, split=-1, n_samples=X.shape[0], gain=gain)
    
    # Check stooping criteria
    if gain < min_gain:
        return tree
    if max_depth is not None:
        if level >= max_depth:
            return tree   
    
    # No stooping criteria was meet, then continue to create the partition
    filter_l = H.iloc[:, j] < split
    H_l, y_l = H.loc[filter_l], y.loc[filter_l]
    H_r, y_r = H.loc[~filter_l], y.loc[~filter_l]
    tree['split'] = [j, split]

    # Next iteration to each split
    
    tree['sl'] = tree_grow(H_l, y_l, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    tree['sr'] = tree_grow(H_r, y_r, level + 1, min_gain=min_gain, max_depth=max_depth, num_pct=num_pct)
    
    return tree

In [98]:
tree_grow(X, y, level=0, min_gain=0.001, max_depth=2, max_features=7, num_pct=10)

{'y_pred': 1,
 'y_prob': 0.5795316301703163,
 'level': 0,
 'split': [0, 51704.54545454545],
 'n_samples': 13150,
 'gain': 0.23348567756020572,
 'sl': {'y_pred': 1,
  'y_prob': 0.8377538829151733,
  'level': 1,
  'split': [2, 32632.272727272724],
  'n_samples': 8368,
  'gain': 0.01900022307321969,
  'sl': {'y_pred': 1,
   'y_prob': 0.9266316250547525,
   'level': 2,
   'split': -1,
   'n_samples': 4564,
   'gain': 0.0047073682911970005},
  'sr': {'y_pred': 1,
   'y_prob': 0.7309511297950604,
   'level': 2,
   'split': -1,
   'n_samples': 3804,
   'gain': 0.016863764337785192}},
 'sr': {'y_pred': 0,
  'y_prob': 0.12771739130434784,
  'level': 1,
  'split': [2, 68279.36363636363],
  'n_samples': 4782,
  'gain': 0.0335095760485582,
  'sl': {'y_pred': 0,
   'y_prob': 0.3392036753445635,
   'level': 2,
   'split': -1,
   'n_samples': 1304,
   'gain': 0.011550066769918432},
  'sr': {'y_pred': 0,
   'y_prob': 0.048563218390804595,
   'level': 2,
   'split': -1,
   'n_samples': 3478,
   'gain':

In [99]:
Z = np.random.choice(range(X.shape[1]),3, replace = 0)
X.iloc[: ,Z]

,M_Camry4dr,Mileage,M_Camry
15,0,29242,0
47,0,26465,0
85,1,46739,0
141,0,41722,0
226,0,77669,0
244,0,18963,0
258,0,15063,0
333,0,46001,0
366,0,26609,0
452,1,87621,0


# Exercise 8.2

Estimate a Bagging of 10 Decision Tree Classifiers Manually using the code created in the Notebook #7

Evaluate the accuracy on the testing set

# Exercise 8.3

Implement the variable max_features on the Decision Tree Classifier created in 11.1.

Compare the impact in the results by varing the parameter max_features

Evaluate the accuracy on the testing set

# Exercise 8.4

Estimate a Bagging of 10 Decision Tree Classifiers with `max_features = log(n_features)`

Evaluate the accuracy on the testing set

# Exercise 8.5

Using sklearn, train a RandomForestClassifier

Evaluate the accuracy on the testing set

# Exercise 8.6

Find the best parameters of the RandomForestClassifier (max_depth, max_features, n_estimators)

Evaluate the accuracy on the testing set

# Exercise 8.7 

Using xgboost train a XGBClassifier 

Evaluate the accuracy on the testing set

# Exercise 8.8

Using xgboost train a XGBClassifier 

Modify the parameters learning rate, gamma, colsample_bytree. Explain what each parameter means.

Evaluate the accuracy on the testing set